# 라이브러리 install

In [1]:
! pip install langchain
! pip install pypdf
! pip install openai
! pip install gdown
! pip install chromadb
! pip install tiktoken
! pip install pillow

### 사용할 패키지 불러오기

In [2]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from glob import glob
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pypdf import PdfReader
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import RetrievalQAWithSourcesChain
import gdown
import os
from PIL import Image
import requests
from typing import List, Tuple, Any, Union
from langchain.schema import AgentAction, AgentFinish
from langchain.agents import Tool, AgentExecutor, BaseMultiActionAgent
from langchain.agents.agent import BaseSingleActionAgent


### OpenAI api key 환경 변수 저장

In [3]:
os.environ['OPENAI_API_KEY'] = 'Using-your-openAI api key'

### PDF 불러오기 및 전처리

In [11]:
file_name = "dndCreatear.pdf"
loader = PyPDFLoader(file_name)
documents = loader.load()

In [12]:
print(documents)

[Document(page_content='116제12장:.괴물\nD&D 기초 규칙(버전 1.0). 재판매 금지. 개인적 용도로만 출력 및 사용 가능.괴물의 자료 상자\n가고일 Gargoyle\n중형 원소, 혼돈 악\n방어도 15 (자연 갑옷)\n히트 포인트 52 (7d8+21)\n이동속도 30ft, 비행 60ft\n 근력 민첩 건강 지능 지혜 매력\n 15 (+2) 11 (+0) 16 (+3) 6 (-2) 11 (+0) 7 (-2)\n피해 저항 아다만틴이 아니며 비마법적 공격에 의한 타격/참격/관통 피해\n피해 면역 독성\n상태 면역 탈진, 석화, 중독\n감각능력 암시야 60ft, 상시 감지 10\n언어 대지어\n도전지수 2 (450 xp)\n거짓 외관.  가고일이 움직이지 않고 있으면, 보통의 움직이지 않는 석상\n과 구분할 수 없습니다.\n행동\n다중공격.  가고일은 물기 1회, 할퀴기 1회로 총 2회 공격을 가합니다.\n물기. 근접 무기 공격:  명중 +4, 간격 5ft, 목표 하나. 명중시: 5(1d6+2)\n점의 관통 피해.\n할퀴기. 근접 무기 공격:  명중 +4, 간격 5ft, 목표 하나. 명중시: 5(1d6+2)\n점의 참격 피해.\n이 흉폭한 대지 원소의 괴물들은 기괴하고 악마 같은 석상처럼 생\n겼습니다. 가고일은 건물이나 폐허 속에 잠복해 있다가 갑자기 움\n직여 적으로 공격할 때 적들이 보이는 공포를 즐기곤 합니다.\n가지 황폐자 Twig Blight\n소형 식물, 중립 악\n방어도 13 (자연 갑옷)\n히트 포인트 4 (1d6+1)\n이동속도 20ft \n 근력 민첩 건강 지능 지혜 매력\n 6 (-2) 13 (+1) 12 (+1) 4 (-3) 8 (-1) 3 (-4)\n기술 은신 +3\n피해 취약성 화염\n상태 면역 장님, 귀머거리\n감각능력 맹안시야 60ft(너머는 장님으로 취급), 상시 감지 9\n언어 공용어를 이해하지만 말할 수는 없음\n도전지수 1/8 (25 xp)\n거짓 외관.  황폐자가 움직이지 않고 있으면, 보통의 죽은 

In [13]:
output = []
# text 정제
for page in documents:
    text = page.page_content
    text = re.sub(r"(\w+)-\n(\w+)", r"\1\2", text)   # 안녕-\n하세요 -> 안녕하세요
    text = re.sub(r"(?<!\n\s)\n(?!\s\n)", " ", text.strip()) # "인\n공\n\n지능펙\n토리 -> 인공지능펙토리
    text = re.sub(r"\n\s*\n", "\n\n", text) # \n버\n\n거\n\n킹\n -> 버\n거\n킹
    output.append(text)

In [14]:
print(output)

['116제12장:.괴물 D&D 기초 규칙(버전 1.0). 재판매 금지. 개인적 용도로만 출력 및 사용 가능.괴물의 자료 상자 가고일 Gargoyle 중형 원소, 혼돈 악 방어도 15 (자연 갑옷) 히트 포인트 52 (7d8+21) 이동속도 30ft, 비행 60ft  근력 민첩 건강 지능 지혜 매력  15 (+2) 11 (+0) 16 (+3) 6 (-2) 11 (+0) 7 (-2) 피해 저항 아다만틴이 아니며 비마법적 공격에 의한 타격/참격/관통 피해 피해 면역 독성 상태 면역 탈진, 석화, 중독 감각능력 암시야 60ft, 상시 감지 10 언어 대지어 도전지수 2 (450 xp) 거짓 외관.  가고일이 움직이지 않고 있으면, 보통의 움직이지 않는 석상 과 구분할 수 없습니다. 행동 다중공격.  가고일은 물기 1회, 할퀴기 1회로 총 2회 공격을 가합니다. 물기. 근접 무기 공격:  명중 +4, 간격 5ft, 목표 하나. 명중시: 5(1d6+2) 점의 관통 피해. 할퀴기. 근접 무기 공격:  명중 +4, 간격 5ft, 목표 하나. 명중시: 5(1d6+2) 점의 참격 피해. 이 흉폭한 대지 원소의 괴물들은 기괴하고 악마 같은 석상처럼 생 겼습니다. 가고일은 건물이나 폐허 속에 잠복해 있다가 갑자기 움 직여 적으로 공격할 때 적들이 보이는 공포를 즐기곤 합니다. 가지 황폐자 Twig Blight 소형 식물, 중립 악 방어도 13 (자연 갑옷) 히트 포인트 4 (1d6+1) 이동속도 20ft   근력 민첩 건강 지능 지혜 매력  6 (-2) 13 (+1) 12 (+1) 4 (-3) 8 (-1) 3 (-4) 기술 은신 +3 피해 취약성 화염 상태 면역 장님, 귀머거리 감각능력 맹안시야 60ft(너머는 장님으로 취급), 상시 감지 9 언어 공용어를 이해하지만 말할 수는 없음 도전지수 1/8 (25 xp) 거짓 외관.  황폐자가 움직이지 않고 있으면, 보통의 죽은 덤불과 구분할  수 없습니다. 행동 할퀴기. 근접 무기 공격:  명중 +3, 간격 5ft, 목표 하나. 명중시

In [15]:
doc_chunks = []

for line in output:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000, # 최대 청크 길이
        separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""], #  텍스트를 청크로 분할하는 데 사용되는 문자 목록
        chunk_overlap=0, # 인접한 청크 간에 중복되는 문자 수
    )
    chunks = text_splitter.split_text(line)
    for i, chunk in enumerate(chunks):
        doc = Document(
            page_content=chunk, metadata={"page": i, "source": file_name}
        )
        doc_chunks.append(doc)

In [16]:
print(doc_chunks)

[Document(page_content='116제12장:.괴물 D&D 기초 규칙(버전 1.0). 재판매 금지. 개인적 용도로만 출력 및 사용 가능.괴물의 자료 상자 가고일 Gargoyle 중형 원소, 혼돈 악 방어도 15 (자연 갑옷) 히트 포인트 52 (7d8+21) 이동속도 30ft, 비행 60ft  근력 민첩 건강 지능 지혜 매력  15 (+2) 11 (+0) 16 (+3) 6 (-2) 11 (+0) 7 (-2) 피해 저항 아다만틴이 아니며 비마법적 공격에 의한 타격/참격/관통 피해 피해 면역 독성 상태 면역 탈진, 석화, 중독 감각능력 암시야 60ft, 상시 감지 10 언어 대지어 도전지수 2 (450 xp) 거짓 외관.  가고일이 움직이지 않고 있으면, 보통의 움직이지 않는 석상 과 구분할 수 없습니다. 행동 다중공격.  가고일은 물기 1회, 할퀴기 1회로 총 2회 공격을 가합니다. 물기. 근접 무기 공격:  명중 +4, 간격 5ft, 목표 하나. 명중시: 5(1d6+2) 점의 관통 피해. 할퀴기. 근접 무기 공격:  명중 +4, 간격 5ft, 목표 하나. 명중시: 5(1d6+2) 점의 참격 피해. 이 흉폭한 대지 원소의 괴물들은 기괴하고 악마 같은 석상처럼 생 겼습니다. 가고일은 건물이나 폐허 속에 잠복해 있다가 갑자기 움 직여 적으로 공격할 때 적들이 보이는 공포를 즐기곤 합니다. 가지 황폐자 Twig Blight 소형 식물, 중립 악 방어도 13 (자연 갑옷) 히트 포인트 4 (1d6+1) 이동속도 20ft   근력 민첩 건강 지능 지혜 매력  6 (-2) 13 (+1) 12 (+1) 4 (-3) 8 (-1) 3 (-4) 기술 은신 +3 피해 취약성 화염 상태 면역 장님, 귀머거리 감각능력 맹안시야 60ft(너머는 장님으로 취급), 상시 감지 9 언어 공용어를 이해하지만 말할 수는 없음 도전지수 1/8 (25 xp) 거짓 외관.  황폐자가 움직이지 않고 있으면, 보통의 죽은 덤불과 구분할  수 없습니다. 행동 할퀴기. 근접 무기 공격:  명중 

In [17]:
embeddings = OpenAIEmbeddings()
index = Chroma.from_documents(doc_chunks, embeddings)

### Prompt 및 chain 설정

In [18]:
system_template="""To answer the question at the end, use the following context. If you don't know the answer, just say you don't know and don't try to make up an answer.
I want you to act as my Creature expert. It tells you the information about one of Creatures. 

Below is an example.

“Bugbear(Medium humanoid (goblinoid), chaotic evil)

------------------------------------

Armor Class 16 (hide armor, shield) 
Hit Points 27 (5d8 + 5)
Speed 30 ft.

------------------------------------

STR : 15(+2)
DEX : 14(+2)
CON : 13(+1)
INT : 8(-1)
WIS : 11(+0)
CHA : 9(-1)

------------------------------------

Skills Stealth +6, Survival +2
Senses darkvision 60 ft., passive Perception 10 
Languages Common, Goblin
Challenge 1 (200 XP)

------------------------------------

Brute. A melee weapon deals one extra die of its damage when the bugbear hits with it (included in the attack).
Surprise Attack. If the bugbear surprises a creature and hits it with an attack during the first round of combat, the target takes an extra 7 (2d6) damage from the attack.

------------------------------------

Actions
Morningstar. Melee Weapon Attack: +4 to hit, reach 5 ft., one target. Hit: 11 (2d8 + 2) piercing damage.
Javelin. Melee or Ranged Weapon Attack: +4 to hit, reach 5 ft. or range 30/120 ft., one target. Hit: 9 (2d6 + 2) piercing damage in melee or 5 (1d6 + 2) piercing damage at range.
"

you only answer in Korean

{summaries}

""" #  추가 여부 판단

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]
prompt = ChatPromptTemplate.from_messages(messages)


chain_type_kwargs = {"prompt": prompt}
bk_chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0),
    chain_type="stuff",
    retriever=index.as_retriever(),
    chain_type_kwargs=chain_type_kwargs,
    reduce_k_below_max_tokens=True  # 답변 한계 없이 출력
)

ValidationError: 1 validation error for StuffDocumentsChain
__root__
  document_variable_name summaries was not found in llm_chain input_variables: ['question'] (type=value_error)